In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings

warnings.filterwarnings('ignore')

import pickle

from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.stattools import grangercausalitytests

In [8]:
with open('../preprocess_data/pickle_final.pkl', 'rb') as file:
    df = pickle.load(file)

In [9]:
df.columns

Index(['GNI', 'GNI 증감률', '전체_사교육비', '초등학교_사교육비', '중학교_사교육비', '고등학교_사교육비',
       'GDP디플레이터_증감률', '총지수_증감률', '생활물가지수_증감률', '신선식품지수_증감률', '조혼인율', '조이혼율',
       '여성초혼 연령', '국공립 어린이집 비율', '여성경제 활동참가율', '수도권_인구밀도', '부산_인구밀도', '출생아수',
       '초등학교', '중학교', '고등학교', '대학교', '알코올 질환 인구 10만 명당 여성 사망률',
       '교육비 지출전망CSI_부산', '교육비 지출전망CSI_경기', '교육비 지출전망CSI_강원', '교육비 지출전망CSI_전북',
       '물가수준전망CSI_부산', '물가수준전망CSI_경기', '물가수준전망CSI_강원', '물가수준전망CSI_전북',
       '현재생활형편CSI_부산', '현재생활형편CSI_경기', '현재생활형편CSI_강원', '현재생활형편CSI_전북',
       '소비지출전망CSI_부산', '소비지출전망CSI_경기', '소비지출전망CSI_강원', '소비지출전망CSI_전북',
       '향후경기전망CSI_부산', '향후경기전망CSI_경기', '향후경기전망CSI_강원', '향후경기전망CSI_전북',
       '취업기회전망CSI_부산', '취업기회전망CSI_경기', '취업기회전망CSI_강원', '취업기회전망CSI_전북',
       '주택가격지수_전국', '주택가격지수_수도권', '주택가격지수_지방', '가구소득평균', '가임기(15~45세) 여성수',
       '알코올의존', '알코올남용', '니코틴의존', '니코틴금단', '주요우울장애', '기분부전장애', '강박장애', '공황장애',
       '광장공포증', '사회공포증', '범불안장애', '특정공포증'],
      dtype='object')

In [5]:
y = df[['출생아수']]
gni_real = df[['GNI']]
gni_rate = df[['GNI 증감률']]
tutor_tot = df[['전체_사교육비']]
tutor_elem = df[['초등학교_사교육비']]
tutor_mid = df[['중학교_사교육비']]
tutor_high = df[['고등학교_사교육비']]
gdp_rate = df[['GDP디플레이터_증감률']]
cpi_tot = df[['총지수_증감률']]
cpi_daily = df[['생활물가지수_증감률']]
cpi_fresh = df[['신선식품지수_증감률']]
marriage = df[['조혼인율']]
divorce = df[['조이혼율']]
mar_age = df[['여성초혼 연령']]
kin_ratio = df[['국공립 어린이집 비율']]
female = df[['여성경제 활동참가율']]
den_cap = df[['수도권_인구밀도']]
den_bus = df[['부산_인구밀도']]
alcohol = df[['알코올 질환 인구 10만 명당 여성 사망률']]

edu_csi_bus = df[['교육비 지출전망CSI_부산']]
edu_csi_urban = df[['교육비 지출전망CSI_경기']]
edu_csi_gang = df[['교육비 지출전망CSI_강원']]
edu_csi_gun = df[['교육비 지출전망CSI_전북']]

price_csi_bus = df[['물가수준전망CSI_부산']]
price_csi_urban = df[['물가수준전망CSI_경기']]
price_csi_gang = df[['물가수준전망CSI_강원']]
price_csi_gun = df[['물가수준전망CSI_전북']]

living_csi_bus = df[['현재생활형편CSI_부산']]
living_csi_urban = df[['현재생활형편CSI_경기']]
living_csi_gang = df[['현재생활형편CSI_강원']]
living_csi_gun = df[['현재생활형편CSI_전북']]

cust_csi_bus = df[['소비지출전망CSI_부산']]
cust_csi_urban = df[['소비지출전망CSI_경기']]
cust_csi_gang = df[['소비지출전망CSI_강원']]
cust_csi_gun = df[['소비지출전망CSI_전북']]

after_csi_bus = df[['향후경기전망CSI_부산']]
after_csi_urban = df[['향후경기전망CSI_경기']]
after_csi_gang = df[['향후경기전망CSI_강원']]
after_csi_gun = df[['향후경기전망CSI_전북']]

edu_csi_bus = df[['취업기회전망CSI_부산']]
edu_csi_urban = df[['취업기회전망CSI_경기']]
edu_csi_gang = df[['취업기회전망CSI_강원']]
edu_csi_gun = df[['취업기회전망CSI_전북']]

In [6]:
# 예시 데이터 (실제 데이터로 대체해야 함)
data = [y,gni_real,gni_rate,tutor_tot,tutor_elem,tutor_mid,tutor_high,gdp_rate,cpi_tot,cpi_daily,cpi_fresh,marriage,divorce,mar_age,kin_ratio,female,den_cap,den_bus]


stat_data = []
stat_name = []
# non-stationary
nstat_data = []
nstat_name = []
diff1_data = []
diff1_name = []
diff2_data = []
diff2_name = []

# print(len(data),len(name))

for index,var in enumerate(data):
# ADF 검정 수행
    result = kpss(var.dropna())
    adf_statistic = result[0]
    p_value = result[1]

    # print(f'ADF 통계량: {adf_statistic}')
    # print(f'p-value: {p_value}')

    if p_value <= 0.05:
        print(f'KPSS 검정결과 {name[index]} 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.')

        # log = np.log1p(var)
        # log_first = pd.Series(var).diff().dropna()
        log_first = np.log1p(var).diff().dropna()

        result = kpss(log_first)
        adf_statistic = result[0]
        p_value = result[1]

        # print(f'>> ADF 통계량: {adf_statistic}')
        # print(f'>> p-value: {p_value}')

        if p_value <= 0.05:
            print(f'>> KPSS 검정결과 1차분한 {name[index]} 시계열 데이터가 정상성을 가지지 않습니다.')

            log_second = np.log1p(var).diff().diff().dropna()

            result = kpss(log_second)
            adf_statistic = result[0]
            p_value = result[1]

            # print(f'>>>> ADF 통계량: {adf_statistic}')
            # print(f'>>>> p-value: {p_value}')

            if p_value <=0.05:
                print(f'>>>> KPSS 검정결과 2차분한 {name[index]} 시계열 데이터가 정상성을 가지지 않습니다.')
                nstat_data.append(var)
                nstat_name.append(name[index])

            else:
                print(f'>>>> KPSS 검정결과 2차분한 {name[index]} 시계열 데이터가 정상성을 가집니다.')
                diff2_data.append(var)
                diff2_name.append(name[index])

        else:
            print(f'>>>> KPSS 검정결과 1차분한 {name[index]} 시계열 데이터가 정상성을 가집니다.')
            diff1_data.append(var)
            diff1_name.append(name[index])

    else:
        print(f'ADF 검정결과 {name[index]} 시계열 데이터가 정상성을 가집니다.')
        stat_data.append(var)
        stat_name.append(name[index])


    print()

print("정상성 :", stat_name)
print("1차분결과 정상성: ", diff1_name)
print("2차분결과 정상성: ", diff2_name)
print('정상성 안띰 :', nstat_name)

KPSS 검정결과 출생아수 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 출생아수 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 GNI 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>> KPSS 검정결과 1차분한 GNI 시계열 데이터가 정상성을 가지지 않습니다.
>>>> KPSS 검정결과 2차분한 GNI 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 GNI 증감률 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 GNI 증감률 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 전체_사교육비 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 전체_사교육비 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 초등학교_사교육비 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 초등학교_사교육비 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 중학교_사교육비 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>>>> KPSS 검정결과 1차분한 중학교_사교육비 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 고등학교_사교육비 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>> KPSS 검정결과 1차분한 고등학교_사교육비 시계열 데이터가 정상성을 가지지 않습니다.
>>>> KPSS 검정결과 2차분한 고등학교_사교육비 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 GDP디플레이터_증감률 시계열 데이터가 정상성을 가지지 않으므로 차분을 계산합니다.
>> KPSS 검정결과 1차분한 GDP디플레이터_증감률 시계열 데이터가 정상성을 가지지 않습니다.
>>>> KPSS 검정결과 2차분한 GDP디플레이터_증감률 시계열 데이터가 정상성을 가집니다.

KPSS 검정결과 총지수_증감률 시계열 데이터가 정상성을 가

/var/folders/k8/xm42xp_16cs6fgtvp6ymr9l40000gn/T/ipykernel_67850/936493535.py:19: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
/var/folders/k8/xm42xp_16cs6fgtvp6ymr9l40000gn/T/ipykernel_67850/936493535.py:33: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  result = kpss(log_first)
/var/folders/k8/xm42xp_16cs6fgtvp6ymr9l40000gn/T/ipykernel_67850/936493535.py:19: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  result = kpss(var.dropna())
/var/folders/k8/xm42xp_16cs6fgtvp6ymr9l40000gn/T/ipykernel_67850/936493535.py:33: InterpolationWarning: The test statistic is outside of the range of p-values availab

In [ ]:
print("정상성 :", stat_name)
print("1차분결과 정상성: ", diff1_name)
print("2차분결과 정상성: ", diff2_name)
print('정상성 안띰 :', nstat_name)

In [37]:
kpss_result = pd.DataFrame()
kpss_index = list(df.columns[:-5])+['출생아수']
kpss_result.index = kpss_index
kpss_result['정상성'] = ['O' if i in stat_name else "X" for i in kpss_index]
kpss_result['1차분 정상성'] = ['O' if i in diff1_name else "X" for i in kpss_index]
kpss_result['2차분 정상성'] = ['O' if i in diff2_name else "X" for i in kpss_index]
kpss_result['정상성 안띰'] = ['O' if i in nstat_name else "X" for i in kpss_index]
kpss_result

,정상성,1차분 정상성,2차분 정상성,정상성 안띰
GNI,X,X,O,X
GNI 증감률,X,O,X,X
전체_사교육비,X,O,X,X
초등학교_사교육비,X,O,X,X
중학교_사교육비,X,O,X,X
고등학교_사교육비,X,X,O,X
GDP디플레이터_증감률,X,X,O,X
총지수_증감률,X,O,X,X
생활물가지수_증감률,X,X,O,X
신선식품지수_증감률,O,X,X,X


In [33]:
kpss_index

['GNI',
 'GNI 증감률',
 '전체_사교육비',
 '초등학교_사교육비',
 '중학교_사교육비',
 '고등학교_사교육비',
 'GDP디플레이터_증감률',
 '총지수_증감률',
 '생활물가지수_증감률',
 '신선식품지수_증감률',
 '조혼인율',
 '조이혼율',
 '여성초혼 연령',
 '국공립 어린이집 비율',
 '여성경제 활동참가율',
 '수도권_인구밀도',
 '부산_인구밀도',
 '출생아수']